In [10]:
#Load model from file
import pandas as pd
import numpy as np

def loadmodelfromfile(modelname, verbose):
    f = open(modelname+'.txt', 'r')
    f.readline()
    f.readline()
    fx = []
    A = []
    B = []
    p = []
    l = '_'
    while 1:
        l = f.readline().rstrip()
        if l != '': 
            fx.append(l) 
        else: 
            break
    while 1:
        l = f.readline().rstrip()
        if l != '':
            if l.__contains__(','):
                l = l.split(',')
                l = [float(x) for x in l]
                A.append(l)
            else:    
                A.append(float(l))
        else:
            break
    while 1:
        l = f.readline().rstrip()
        if l != '':
            if l.__contains__(','):
                l = l.split(',')
                l = [float(x) for x in l]
                B.append(l)
            else:    
                B.append(float(l))
        else:
            break
    f.readline()
    f.readline()
    for x in f.readline().rstrip().split(','):
        p.append(int(x))
    f.readline()
    e = float(f.readline().rstrip())

    if verbose:
        print("Model: %s" % modelname)
        print("Function list: ")
        print(fx) 
        print("A matrice: ")
        print(A)
        print("B matrice: ")
        print(B)
        print("Priority factor array (most important to least important): ")
        print(p)
        print("Limit for the ratio between the weights ranked first and last: ")
        print(e)

    return fx, A, B, p, e

In [11]:
#Conversion functions
import numpy as np
import math

def lin(lin_y,lin_X,lin_Y):
    lin_m = (lin_Y[1]-lin_Y[0])/(lin_X[1]-lin_X[0])
    lin_b = lin_Y[0] - lin_m*lin_X[0]
    lin_x = (lin_y - lin_b)/lin_m
    return lin_x

def converttovaluescale(el,uf_i,uf_m):
    match uf_m:
        case 'lin':
            reverse_A = A[uf_i].copy()
            reverse_A.sort(reverse=True)
            index = A[uf_i].index(min(reverse_A,key=lambda x : x - el > 0 ))
            A_uf_i = np.array(A[uf_i])
            B_uf_i = np.array(B[uf_i])
            if index == len(A[uf_i])-1:
                return B_uf_i[index]
            else:
                return (lin(el,B_uf_i[index:index+2],A_uf_i[index:index+2])) 
        case 'exp':
            return A[uf_i]*math.exp(el)+B[uf_i]
        case 'log':
            return A[uf_i]*math.log(el)+B[uf_i]
        case _:
            return -999

#Convert data to value scale
def convertoriginaltovaluescale(modelname, fx, A, B):
    #Read data
    df1 = pd.read_csv(modelname+'_originals.csv')
    #print(df1.head(100))

    #Convert
    n = len(fx)
    for i in range(1,n+1):
        df1.iloc[:, i] = df1.iloc[:, i].apply(lambda x: converttovaluescale(x,i-1,fx[i-1]))
    #print(df1.head(10))
    #Save data
    df1_headers = ['DMUs']
    for i in range(1,n+1):
        df1_headers.append('V'+str(i))
    df1.columns = df1_headers
    df1.to_csv(modelname+"_valuescale.csv",float_format='%.12f', index=False)
    return df1

In [12]:
import pandas as pd
modelname = 'ModeloHoje'

#Load mdodel from file
fx = []
A = []
B = []
p = []
fx,A,B,p,e = loadmodelfromfile(modelname, False)
#Convert model from original to value scale 
df_vscale = convertoriginaltovaluescale(modelname, fx,A,B)
print(df_vscale.head())

   DMUs        V1        V2        V3        V4        V5        V6
0     1  0.957958  0.883918  0.832385  0.417558  0.429613  0.550340
1     2  0.968563  0.906813  0.886381  0.557093  0.133438  0.355864
2     3  0.093579  0.107303  0.105978  0.584673  0.422685  0.277777
3     4  0.940571  0.849797  0.794183  0.447082  0.020103  0.212868
4     5  0.887684  0.762808  0.813892  0.348892  0.646858  0.610541


In [13]:
#Phase one with no restrictions
import numpy as np
import pandas as pd
from scipy.optimize import linprog

def phase1(modelname,load,df_vscale):
    #Load values
    if load:
        df_vscale = pd.read_csv(modelname+'_valuescale.csv') 
    df_vscale2 = df_vscale.iloc[:,1:]
    nw = len(df_vscale2.columns)
    ndmu = len(df_vscale2)
    _A = df_vscale2.to_numpy()
    #print(_A)
    #Run Solver
    res_fase1 = []
    for i in range(ndmu):
        #Configurar um array para guardar as varaiaveis de decisao e coeficentes das mesmas na fo (participam todos os w e o d)
        #c = np.array([0,0,0,0,0,0,1])
        c0 = np.zeros(nw)
        c = np.append(c0,1)

        #Configuracao das restricoes
        A2 = np.copy(_A)
        A_27 = np.copy(_A[i])
        A2[i] = np.zeros(nw)
        A3 = A2 - A_27.T

        #Configuracao das restricoes <=
        #Coeficentes
        A_ub = np.c_[A3,-1*np.ones((ndmu,1))]
        #Valores
        b_ub= np.zeros(ndmu)

        #Configuracao das restricoes =
        #A_eq = [np.array([1,1,1,1,1,1,0])]
        A_eq0 = np.ones(nw)
        A_eq = [np.append(A_eq0,0)]
        #Valores
        b_eq = [1]

        #Configurar um array com os limites das variaveis de decisao (ex. > 0)
        wn_bounds = []
        for j in range(nw):
            wn_bounds.append((0,None))
        wn_bounds.append((None,None))

        res = linprog(c, A_ub=A_ub, b_ub=b_ub,A_eq=A_eq, b_eq=b_eq,bounds=wn_bounds)
        #print('Optimal value:', res.fun, '\nX:', res.x)
        res2 = np.zeros(nw+2)
        res2[0] = i+1
        res2[1:] = res.x
        res_fase1.append(res2)
    
    #Save data to file
    column_names = []
    column_names.append('DMUs')
    for i in range(1,nw+1):
        column_names.append('w'+str(i))
    column_names.append('d*')
    df2 = pd.DataFrame(data = res_fase1, columns=column_names)

    d_col = df2.pop('d*')
    df2.insert(1,"d*",d_col)
    #print(df2.head(10))
    df2.to_csv(modelname+"_phase1.csv", float_format='%.12f', index = False)
    return df2

#Run model
modelname = 'ModeloHoje'
df_phase1 = phase1(modelname,False,df_vscale)
print(df_phase1.head(10))

   DMUs        d*            w1            w2            w3            w4  \
0   1.0  0.032778  9.354732e-01  1.535428e-12  3.098436e-13  6.452685e-02   
1   2.0  0.013854  9.354732e-01  1.508783e-10  1.306686e-10  6.452685e-02   
2   3.0  0.251101  3.811616e-11  3.738867e-11  6.112567e-11  7.692710e-01   
3   4.0  0.047138  9.354732e-01  1.342757e-11  4.033977e-12  6.452685e-02   
4   5.0  0.079405  8.585148e-01  2.172903e-12  9.506447e-12  8.398669e-05   
5   6.0 -0.047815  1.167652e-12  9.324337e-01  4.096968e-13  6.878314e-13   
6   7.0  0.040496  8.586251e-01  2.447048e-10  4.451480e-10  1.091959e-10   
7   8.0  0.004143  8.586251e-01  2.187410e-09  1.163968e-10  2.525973e-10   
8   9.0  0.036762  8.347124e-01  6.851958e-13  5.198658e-13  1.652876e-01   
9  10.0  0.124520  8.585148e-01  1.501460e-09  4.356023e-09  8.398697e-05   

             w5            w6  
0  3.857959e-13  4.330606e-13  
1  3.309367e-11  3.337232e-11  
2  2.307290e-01  3.365787e-11  
3  1.644217e-12  6.99132

In [51]:
#Phase two with no restrictions
import numpy as np
import pandas as pd
from scipy.optimize import linprog

def phase2(modelname,load,df_vscale,df_phase1):
    #Load values
    if load:
        df_values = pd.read_csv(modelname+'_valuescale.csv')
        df_phase1 = pd.read_csv(modelname+'_phase1.csv')
    

    #81 lambdas -> maiores que zero e o somatorio dos 81 lambdas tem de ser igual a 1
    #6 slacks -> todas maiores que zero 
    #d tem de entrar

    #Colocar primeiro as slacks e depois os lambdas
    df_vscale2 = df_vscale.iloc[:,1:]
    nw = len(df_vscale2.columns)
    ndmu = len(df_vscale2)
    #Copiar o array com os valores dos vs
    _vs = df_vscale2.to_numpy()

    #Copiar os ws
    df_phase1_2 = df_phase1.iloc[:,2:]
    _ws = df_phase1_2.to_numpy()
   
    #Run Solver
    res_fase2 = []
    
    for k in range(ndmu):
        print("k",k)
        #Configurar um array para guardar as varaiaveis de decisao e coeficentes das mesmas na fo (participam todos os w e o d)
        ck = -1*_ws[k] #coeficientes das slacks
        ck = np.append(ck,np.zeros(ndmu)) #lambdas sem coeficiente

        A_eq = []
        b_eq = []
        A_eq0 = np.zeros(nw)
        A_eq0 = np.append(A_eq0,np.ones(ndmu))
        A_eq0[nw+k] = 0
        A_eq.append(A_eq0.tolist())
        b_eq.append(1)

        A_eq_dmu = np.zeros(nw+ndmu)
        A_eq_dmu[nw+k] = 1
        A_eq.append(A_eq_dmu.tolist())
        b_eq.append(0)

        for c in range(nw):
            lj = np.ones(ndmu)
            lj = lj * _vs[:,c]
            lj[k] = 0
            sj = np.zeros(nw)
            sj[c] = -1
            A_eq_c = np.append(sj,lj)
            A_eq.append(A_eq_c)
            b_eq.append(_vs[k][c])
            #Restricao a propria dmu cujo lambda deve ser 0
        #print(A_eq)
        #print(b_eq)
        
        res = linprog(ck, A_eq=A_eq, b_eq=b_eq)
        print('Optimal value:', res.fun, '\nX:', res.x)
        #print('Optimal value:', res.fun, '\nX:', res.x)
        res2 = np.zeros(nw+ndmu+2)
        res2[0] = k+1
        res2[1] = -1*res.fun
        res2[2:] = res.x
        res_fase2.append(res2)
    
    #Save data to file
    column_names = []
    column_names.append('DMUs')
    column_names.append('d*')
    for i in range(1,nw+1):
        column_names.append('s'+str(i))
    for i in range(1,ndmu+1):
        column_names.append('l'+str(i))
    
    df2 = pd.DataFrame(data = res_fase2, columns=column_names)
    #d_col = df2.pop('d*')
    #df2.insert(1,"d*",d_col)
    #print(df2.head(10))
    df2.to_csv(modelname+"_phase2.csv", float_format='%.12f', index = False)
    return df2
    
#Run model
modelname = 'ModeloHoje'
df_phase2 = phase2(modelname,True,df_vscale,df_phase1)
#print(df_phase2.head(10))

k 0
Optimal value: -0.032778101923549775 
X: [2.99546261e-02 7.25511763e-02 1.00124382e-01 7.37112317e-02
 1.70181394e-01 8.90741074e-02 0.00000000e+00 5.88463628e-11
 4.63007350e-11 2.96990223e-11 4.18172400e-11 6.17464077e-01
 4.91003682e-11 6.01259790e-11 6.73074504e-11 3.04992705e-11
 4.31393051e-11 5.53640499e-11 3.38305735e-11 3.79725429e-11
 5.42368730e-11 5.69140887e-11 4.06734183e-11 3.44360744e-11
 4.20552802e-11 4.89985392e-11 3.82535921e-01 3.98483560e-11
 3.65563538e-11 2.72796902e-11 8.86954970e-11 3.41460089e-11
 3.41134755e-11 1.72892389e-10 3.73207539e-11 3.25911135e-11
 6.11565605e-11 2.11775371e-11 3.09493516e-11 1.13119864e-10
 7.43233057e-11 4.21762112e-11 4.53167953e-11 1.11401215e-10
 5.42686795e-11 3.83704033e-11 1.25749974e-10 4.69659630e-11
 6.28617402e-11 4.94594641e-11 4.12362899e-11 3.97368690e-10
 7.90675573e-11 3.84652576e-11 5.55628180e-11 1.47515353e-10
 5.21041836e-11 2.20559858e-11 4.46079865e-11 4.28877236e-11
 2.81858598e-11 7.49445912e-11 5.4793012

In [120]:
#Convert to original scales
from itertools import count
import numpy as np
import pandas as pd
import math

def lin_1(lin_y,lin_X,lin_Y):
    lin_m = (lin_Y[1]-lin_Y[0])/(lin_X[1]-lin_X[0])
    lin_b = lin_Y[0] - lin_m*lin_X[0]
    lin_x = (lin_y - lin_b)/lin_m
    return lin_x
    
def converttooriginalscale(el,uf_i,uf_m):
    
    if pd.isna(el): return el
    match uf_m:
        case 'lin':
            #print(B[uf_i])
            #Verificar se o B esta por ordem crescente
            if B[uf_i][0]>B[uf_i][1]:
                A[uf_i].sort(reverse=False)
                B[uf_i].sort(reverse=False)
            
            reverse_B = B[uf_i].copy()
            reverse_B.sort(reverse=True)            
            index = B[uf_i].index(min(reverse_B,key=lambda x : x - el > 0 ))

            A_uf_i = np.array(A[uf_i])
            B_uf_i = np.array(B[uf_i])
            
            if index == len(B[uf_i])-1:
                return A_uf_i[index]
            else:
                return (lin_1(el,A_uf_i[index:index+2],B_uf_i[index:index+2]))
        
        case 'exp':
            return math.log((el-B[uf_i])/A[uf_i])
        case 'log':
            return math.exp((el-B[uf_i])/A[uf_i])
        case _:
            return -999


#Convert data to value scale
def convertvaluescaletooriginal():
    #Read data
    df = pd.read_csv(modelname+'_phase2.csv')
    #print(df)
    #df = df.iloc[:,1:]
    #Convert
    #print(df.head(10))
    
    n = len(fx)
    #Valores w
    for i in range(1,n+1):
        #print("f1:",i)
        df.iloc[:, i+1] = df.iloc[:, i+1].apply(lambda x: converttooriginalscale(x,i-1,fx[i-1]))
    #Valores das slacks
    for i in range(n+1,2*n+1):
        df.iloc[:, i+1] = df.iloc[:, i+1].apply(lambda x: converttooriginalscale(x,i-1-(n+1),fx[i-1-(n+1)]))

    #print(df.head(10))
    #Save data
    df.to_csv(modelname+"_final.csv", float_format='%.12f',index=False)
    #print(df.head())

#Convert model from value to original scale
import pandas as pd
modelname = 'Model2'

#Load mdodel from file
fx = []
A = []
B = []
p = []
fx,A,B,p,e = loadmodelfromfile(modelname, False)

df_final = convertvaluescaletooriginal()
print(df_final)

f1: 6
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5     0.136
6       NaN
7     0.073
8       NaN
9     0.020
10    0.036
11    0.063
Name: S1, dtype: float64
f1: 7
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5     0.142
6       NaN
7     0.000
8       NaN
9     0.329
10    0.094
11    0.079
Name: S2, dtype: float64
f1: 8
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5     0.215
6       NaN
7     0.221
8       NaN
9     0.060
10    0.058
11    0.213
Name: S3, dtype: float64
f1: 9
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
5     0.015
6       NaN
7     0.371
8       NaN
9     0.056
10    0.162
11    0.040
Name: S4, dtype: float64
f1: 10
0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     0.0
6     NaN
7     0.0
8     NaN
9     0.0
10    0.0
11    0.0
Name: S5, dtype: float64
None


In [121]:
#Load
df_final = pd.read_csv(modelname+'_final.csv') 
df_final.head(15)

,DMU,d*,w1,w2,w3,w4,w5,S1,S2,S3,S4,S5,DMU1,DMU2,DMU4,DMU5,DMU7
0,1,-0.0339,1.246120e+06,2.422214e+06,2.976351e+06,50000.0,17208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,-0.0226,1.624885e+06,3.419780e+06,2.327958e+06,50000.0,19152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,-0.0115,1.624885e+06,1.068013e+06,2.976351e+06,93120.0,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,-0.0420,9.996554e+05,5.451318e+06,1.041331e+06,50000.0,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,-0.0740,1.519459e+05,5.451318e+06,2.976351e+06,50000.0,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,0.0504,4.445151e+05,1.896444e+06,1.298367e+06,295000.0,11936.0,8352.0,1.160608e+06,3.602396e+06,2.917955e+06,50000.0,0.000,0.000,0.698,0.000,0.302
6,7,-0.1535,1.624885e+06,5.451318e+06,2.976351e+06,50000.0,25000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,0.0238,5.907212e+05,2.290581e+06,1.662189e+06,254200.0,14016.0,6336.0,1.624885e+06,3.560990e+06,1.823223e+06,50000.0,0.059,0.000,0.000,0.422,0.518
8,9,-0.1672,1.624885e+06,5.451318e+06,2.264265e+06,434400.0,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,0.0240,5.782563e+05,1.567104e+06,1.246272e+06,337600.0,11424.0,4640.0,7.451397e+05,4.856172e+06,2.764119e+06,50000.0,0.000,0.790,0.000,0.000,0.210


In [8]:
#Convert model from original to utility scale
modelname = 'ModeloHoje'

#Convert model from 
fx = []
A = []
B = []
p = []
fx,A,B,p,e = loadmodelfromfile(True)
convertoriginaltovaluescale()

#df1 = pd.read_csv(modelname+'_originals.csv')

Model: ModeloHoje
Function list: 
['lin', 'lin', 'lin', 'lin', 'lin', 'lin']
A matrice: 
[[6.95, 243.5314366], [8.66, 56.69757456], [0.48, 3.37886725], [0.18, 1.0880749], [0.19, 1.25329765], [-2.25, 3.17501118]]
B matrice: 
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0]]
Priority factor array (most important to least important): 
[1, 2, 3, 4, 5, 6]
Limit for the ratio between the weights ranked first and last: 
5.32


IndexError: single positional indexer is out-of-bounds

In [38]:
#Run model
modelname = 'Model2'

phase1()



[[0.6326 0.8237 0.572  0.7435 0.1234]
 [0.5293 0.7889 0.7883 0.7335 0.1016]
 [0.5266 0.8242 0.4747 0.7851 0.0702]
 [0.7464 0.7354 0.8247 0.6866 0.1028]
 [0.8204 0.8088 0.7527 0.708  0.051 ]
 [0.411  0.4063 0.4133 0.5518 0.2476]
 [0.0838 0.1153 0.173  0.3    0.5847]
 [0.3547 0.4503 0.2201 0.1361 0.3338]
 [0.6936 0.7871 0.7854 0.9449 0.1009]
 [0.4161 0.318  0.5992 0.5782 0.2265]
 [0.2015 0.2526 0.3264 0.2857 0.3969]
 [0.35   0.3469 0.2194 0.4518 0.3458]]
[ 1.00000000e+00  7.35974305e-03  4.67597831e-01  3.22100028e-11
  3.80525911e-11  5.25042426e-01 -2.84785909e-02]
[ 2.00000000e+00  4.06521270e-11  4.31868154e-01  5.68131845e-01
  1.06658747e-11  4.85940482e-10 -2.42494503e-03]
[ 3.00000000e+00  1.18438111e-11  8.46218487e-01  2.74366543e-11
  1.53781513e-01  1.65629182e-11 -6.82042016e-03]
[ 4.00000000e+00  2.05015666e-01  2.83339558e-09  7.94984331e-01
  1.14421809e-10  6.15665703e-10 -4.20677112e-02]
[ 5.00000000e+00  9.99999999e-01  3.22811511e-10  1.43074826e-10
  8.65222897e-10  

In [3]:
import numpy as np

a = np.array([3,4,5])
print(a)
print(np.flip(a))

[3 4 5]
[5 4 3]
